In [1]:
import numpy as np
import pandas as pd
from tqdm import tqdm_notebook as tqdm
from sklearn.externals import joblib
%matplotlib inline
import seaborn as sns
import gc
import os, sys

In [2]:
path = './model_performance/tmp/'
dirs = os.listdir( path )
valid_files = (list(path + file for file in dirs if 'valid' in file))
valid_files

['./model_performance/tmp/valid_v4.csv',
 './model_performance/tmp/valid_v3.csv']

In [3]:
test_files = (list(path + file for file in dirs if 'pred' in file))
test_files

['./model_performance/tmp/v3_pred_0.788.csv',
 './model_performance/tmp/v4_pred_0.793.csv']

In [4]:
stack = pd.concat([pd.read_csv(file).rename(index=str, columns={"pred": "pred_"+ str(i)}) for i,file in enumerate(valid_files)],axis=1)
_, i = np.unique(stack.columns, return_index=True)
stack = stack.iloc[:, i]
stack.head()

,Target,pred_0,pred_1
0,1.0,0.817677,0.817677
1,0.0,0.182188,0.182188
2,0.0,0.335287,0.335287
3,0.0,0.332494,0.332494
4,0.0,0.436043,0.436043


In [5]:
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.cross_validation import train_test_split
from sklearn import metrics

X = stack.loc[:, stack.columns != 'Target']
y = stack['Target']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

/home/lizihaoleo/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [22]:
# columns = []
# for i in range(10):
#     column = ('%.1fv1_%.1fv2'%(i*.1, 1-i*.1))
#     columns.append(column)
#     stack[column] = i*.1*stack['v1'] + (1-i*.1)*stack['v2']
    
# stack.head()

In [23]:
# result = {}
# for column in columns:
#     pred = stack[column]
#     result[column] = metrics.roc_auc_score(stack['target'], pred)
# sorted(result.items(), key=lambda x: x[1], reverse= True)

In [6]:
ensembler = LogisticRegression()
ensembler.fit(X_train,y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [7]:
y_pred = ensembler.predict_proba(X_test)[:,1]
# y_pred
metrics.roc_auc_score(y_test,y_pred)

0.796095395508031

In [8]:
stack_pred = pd.concat([pd.read_csv(file).rename(index=str, columns={"TARGET": "pred_"+ str(i)}) for i,file in enumerate(test_files)],axis=1)
_, i = np.unique(stack_pred.columns, return_index=True)
stack_pred = stack_pred.iloc[:, i]
stack_pred.head()

,SK_ID_CURR,pred_0,pred_1
0,100001,0.271999,0.271999
1,100005,0.593314,0.593314
2,100013,0.188375,0.188375
3,100028,0.229896,0.229896
4,100038,0.675568,0.675568


In [9]:
stack_pred['TARGET'] = ensembler.predict_proba(stack_pred.drop(['SK_ID_CURR'],axis=1))[:,1]
stack_pred.head()

,SK_ID_CURR,pred_0,pred_1,TARGET
0,100001,0.271999,0.271999,0.034947
1,100005,0.593314,0.593314,0.144875
2,100013,0.188375,0.188375,0.023662
3,100028,0.229896,0.229896,0.028733
4,100038,0.675568,0.675568,0.200948


In [10]:
submit = stack_pred[['SK_ID_CURR','TARGET']]
submit.to_csv('./model_performance/stack_model.csv',index=False)

In [11]:
# just a test
submit['TARGET'] = stack_pred.drop(['SK_ID_CURR','TARGET'],axis=1).mean(axis=1)
submit.to_csv('./model_performance/stack_model_mean.csv',index=False)

/home/lizihaoleo/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
